In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score

# Load recommendation CSV (your friend's result)
recommended_df = pd.read_csv("/content/results_w_pos(Singapore).csv")

# Load the test data
with open('/content/singapore-keywords_test.json', 'r') as file:
    test_data = json.load(file)

print("Data loaded successfully.")


Data loaded successfully.


In [ ]:
# Extract ground truth from np2rests and np2users
user_ground_truth = {}
np2rests = test_data['np2rests']
np2users = test_data['np2users']

for keyword, restaurants in np2rests.items():
    users = np2users.get(keyword, [])
    for user in users:
        user_ground_truth.setdefault(user, set()).update(restaurants.keys())

# Convert sets to lists for easier processing
user_ground_truth = {user: list(rests) for user, rests in user_ground_truth.items()}

# Verify extraction
for user, rests in list(user_ground_truth.items())[:3]:
    print(f"User: {user}, Restaurants visited: {rests}")


User: H8mXfh5XgGCqmMLwVH7k5A, Restaurants visited: ['8I8ZfuOXNPfOEL3ijjsOPg', 'Rs94MowK7qrq5O96cILpgg', 'nCwEzk4Z1lX5PvLEYpbBHw', 'O79uzVop5yqobf3bdjZzQg', 'XSw22_2mEntgX8I4tXBEhQ', '0PuEldv2_G0AoBwIXeXMDA', '9R5laAm5Kv4aBLVtH1gVnA', 'bOE1ZFgV04cquL5DRbf_PA', 'eawsC19ruMWUv0FsPMR3QA', '7E8bLS3-3HxI-TdyfCz-_g', 'xsaHJx_tkVj1RArC2Fr3PA', 'y8SKD6zMFpBhUpujG3grOQ', 'qCAecmAZl8DIeXMmEbWjCw', 'Q3e9a5Tv3IMXw9D7SATL5w', 'S28wsa8TvT_ptmbxVuvzTA', 'Z0JiIjgjactv8LqSoH2-yA', 'Gi3VGQT0zRrdUEQrfYPbUg', 'z4qzeRcds-nntmWlJ8K6RA', 'CdAmgWWCabZQoYNNWOKcEw', 'KEoj9uw6orK0ihhpRfxzlA', '2FKst9yfxxoQAzPkJ6RNrQ', 'z4cZR4JeMWvlXMtxirWBGg', 'ZV4pEd7DWJ6JhYRulGZbgw', 'Xtr2M_mh9_jOkmpNd_CSCA', '-uwbTW9ccBvpjGJX5s7x7w', 'LRPfJV_BAIz_UWGKIupiGQ', '6AegFR-Lx8897VfiS2Ey0g', 'DQ8aBXrsVrZ4bPTHUQZBng', 'jSPujFthkNYpBwnOLbiNEg', 'io2XkNYPBCqOiBeRX8lNdQ', '0e30ReHOmL2GzY_3hEy_Ig', 'VzRMui5cpdzkqYvHaHy5dw', 'l5lEOq47ne-jWKl67VKwZA', 'jx13gTcw7-HTd1IBqaY4xw', 'tq03doSUN1Hpd2zmNAMRCQ', 'Z9k90aXeLw2LAD3p0EvHGg', 'VZcs1K7A31s

In [ ]:
# Quick verification
sample_user = recommended_df.iloc[0]['user']
print("Recommended restaurants:", recommended_df.iloc[0]['restaurant_names'])
print("Ground truth restaurants:", user_ground_truth.get(sample_user))


Recommended restaurants: fY1IkBnRft1KR0O2tqu7pg, MPzZuWpKpeLBa833zzL9IQ, 2tGFUtUrE0DhwlX59pbArA, izEBByeNB835I5makSbdew, fxQKJjAs7sayW3Zg45kE_g, -vVXB-MBTTa8EUmJhDafQQ, KsHdXhRdu9UzjZXO7g9BHg, ahlZC7XOKN-XJ7P4sPZrRg, N8KDtnIu6x0uQ-JogBNeAg, MqdAYfl1p_c1144i48k6jw
Ground truth restaurants: ['8I8ZfuOXNPfOEL3ijjsOPg', 'Rs94MowK7qrq5O96cILpgg', 'nCwEzk4Z1lX5PvLEYpbBHw', 'O79uzVop5yqobf3bdjZzQg', 'XSw22_2mEntgX8I4tXBEhQ', '0PuEldv2_G0AoBwIXeXMDA', '9R5laAm5Kv4aBLVtH1gVnA', 'bOE1ZFgV04cquL5DRbf_PA', 'eawsC19ruMWUv0FsPMR3QA', '7E8bLS3-3HxI-TdyfCz-_g', 'xsaHJx_tkVj1RArC2Fr3PA', 'y8SKD6zMFpBhUpujG3grOQ', 'qCAecmAZl8DIeXMmEbWjCw', 'Q3e9a5Tv3IMXw9D7SATL5w', 'S28wsa8TvT_ptmbxVuvzTA', 'Z0JiIjgjactv8LqSoH2-yA', 'Gi3VGQT0zRrdUEQrfYPbUg', 'z4qzeRcds-nntmWlJ8K6RA', 'CdAmgWWCabZQoYNNWOKcEw', 'KEoj9uw6orK0ihhpRfxzlA', '2FKst9yfxxoQAzPkJ6RNrQ', 'z4cZR4JeMWvlXMtxirWBGg', 'ZV4pEd7DWJ6JhYRulGZbgw', 'Xtr2M_mh9_jOkmpNd_CSCA', '-uwbTW9ccBvpjGJX5s7x7w', 'LRPfJV_BAIz_UWGKIupiGQ', '6AegFR-Lx8897VfiS2Ey0g', 'DQ8aBX

In [ ]:
# Define evaluation functions
def precision_recall_at_k(actual, recommended, k):
    recommended_at_k = recommended[:k]
    hits = len(set(actual) & set(recommended_at_k))
    precision = hits / k
    recall = hits / len(actual) if actual else 0
    return precision, recall

def average_precision(actual, recommended, k):
    recommended_at_k = recommended[:k]
    score = 0
    hits = 0
    for i, p in enumerate(recommended_at_k):
        if p in actual:
            hits += 1
            score += hits / (i + 1)
    return score / min(len(actual), k) if actual else 0

# Initialize metric containers
k_values = [5, 10, 20]
precision_scores = {k: [] for k in k_values}
recall_scores = {k: [] for k in k_values}
avg_precision_scores = []

# Calculate metrics
for idx, row in recommended_df.iterrows():
    user = row['user']
    recommended = row['restaurant_names'].split(", ")
    actual = user_ground_truth.get(user, [])

    for k in k_values:
        precision, recall = precision_recall_at_k(actual, recommended, k)
        precision_scores[k].append(precision)
        recall_scores[k].append(recall)

    avg_precision_scores.append(average_precision(actual, recommended, 20))

# Display averages clearly
for k in k_values:
    avg_precision = np.mean(precision_scores[k])
    avg_recall = np.mean(recall_scores[k])
    print(f'Precision@{k}: {avg_precision:.4f}')
    print(f'Recall@{k}: {avg_recall:.4f}')

MAP_score = np.mean(avg_precision_scores)
print(f'Mean Average Precision (MAP): {MAP_score:.4f}')


Precision@5: 0.8132
Recall@5: 0.0068
Precision@10: 0.7970
Recall@10: 0.0133
Precision@20: 0.3985
Recall@20: 0.0133
Mean Average Precision (MAP): 0.3488


In [ ]:
# Step 4: Calculate Recommendation Coverage
unique_recommended = set()
for restaurants in recommended_df['restaurant_names']:
    unique_recommended.update(restaurants.split(", "))

# Total restaurants from the original dataset
total_restaurants = len(set().union(*test_data['np2rests'].values()))

coverage = len(unique_recommended) / total_restaurants * 100
print(f'Recommendation Coverage: {coverage:.2f}%')


Recommendation Coverage: 46.09%
